In [1]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
compare_folder = "/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2020-11-23_COVID"
pairwise_filename = [x for x in os.listdir(compare_folder) if '.snp.pairwise.tsv' in x][0]
pairwise_file = os.path.join(compare_folder, pairwise_filename)
pwdf = pd.read_csv(pairwise_file, sep="\t" , names=['sample_1', 'sample_2', 'dist'])

In [3]:
pwdf.head()

,sample_1,sample_2,dist
0,20069944,20069944,0
1,20069944,20069551,16
2,20069944,20091271,2
3,20069944,20081614,12
4,20069944,20267853,9


In [4]:
pwdf['sample_1'] = pwdf['sample_1'].astype(str)

In [5]:
pwdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833569 entries, 0 to 833568
Data columns (total 3 columns):
sample_1    833569 non-null object
sample_2    833569 non-null object
dist        833569 non-null int64
dtypes: int64(1), object(2)
memory usage: 19.1+ MB


In [6]:
samples = list(set(pwdf.sample_1.tolist()))

In [7]:
len(samples)

913

In [8]:
samples_type = [type(x) for x in samples]
dict((x,samples_type.count(x)) for x in set(samples_type))

{str: 913}

In [9]:
wgs = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/WGS_COVID.csv')

In [10]:
wgs.head()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
0,20060161,09/03/2020,174.0,Madrid,H,84,9,3,2020
1,20060178,09/03/2020,175.0,Madrid,H,76,9,3,2020
2,20062236,10/03/2020,176.0,Madrid,M,96,10,3,2020
3,20062563,10/03/2020,177.0,Madrid,H,78,10,3,2020
4,20063795,12/03/2020,178.0,Madrid,H,85,12,3,2020


In [11]:
wgs.shape

(1020, 9)

In [12]:
wgs['id'] = wgs['id'].astype(str)

In [13]:
samples_wgs = wgs.id.tolist()
#samples_wgs = [str(x) for x in samples_wgs]

In [14]:
WGS = set(samples_wgs)
PW = set(samples)

In [15]:
len(PW)

913

In [16]:
missing_wgs = list(PW - WGS)
len(missing_wgs)

130

In [17]:
missing_wgs_type = [type(x) for x in missing_wgs]
dict((x,missing_wgs_type.count(x)) for x in set(missing_wgs_type))

{str: 130}

In [18]:
missing_pw = list(WGS - PW)
len(missing_pw)

143

In [19]:
all_missing = missing_pw + missing_wgs
len(all_missing)

273

In [20]:
wgs.shape

(1020, 9)

In [21]:
df = wgs[~wgs.id.isin(missing_pw)]

In [22]:
df.shape

(871, 9)

In [23]:
df2 = pd.DataFrame(missing_wgs, columns=['id'])

In [24]:
df2['id'] = df2['id'].astype(str)

In [25]:
df2.head()

,id
0,20296883ciclos8
1,20253153
2,20431656
3,20431872
4,20438339


In [26]:
df.head()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
0,20060161,09/03/2020,174.0,Madrid,H,84,9,3,2020
1,20060178,09/03/2020,175.0,Madrid,H,76,9,3,2020
2,20062236,10/03/2020,176.0,Madrid,M,96,10,3,2020
3,20062563,10/03/2020,177.0,Madrid,H,78,10,3,2020
4,20063795,12/03/2020,178.0,Madrid,H,85,12,3,2020


In [27]:
final_df = df.append(df2).reset_index(drop=True)
final_df = final_df[ ['id', 'full_date', 'patient__autocolor', 'province__autocolor', 'SEX__autocolor',
       'AGE__autocolor', 'day', 'month', 'year'] ]

In [32]:
metadata = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/wgs_covid_metadata.csv')
metadata['id'] = metadata['id'].astype(str)

In [33]:
metadata.tail()

,id,DEPARTMENT__autocolor
430,20414120,ONCOLOGIA
431,20414510,DIVERSIDAD
432,20414512,OTORRINOLARINGOLOGIA(I)
433,20414513,MICROBIOLOGIA
434,20421290,URGENCIAS


In [34]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 2 columns):
id                       435 non-null object
DEPARTMENT__autocolor    435 non-null object
dtypes: object(2)
memory usage: 6.9+ KB


In [35]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 9 columns):
id                     1001 non-null object
full_date              871 non-null object
patient__autocolor     788 non-null float64
province__autocolor    871 non-null object
SEX__autocolor         871 non-null object
AGE__autocolor         870 non-null object
day                    871 non-null float64
month                  871 non-null float64
year                   871 non-null float64
dtypes: float64(4), object(5)
memory usage: 70.5+ KB


In [36]:
final_df = final_df.fillna(0)
final_df.day = final_df.day.astype('Int64')
final_df.month = final_df.month.astype('Int64')
final_df.year = final_df.year.astype('Int64')
final_df.replace(0, np.nan, inplace=True)

In [37]:
final_df = final_df.merge(metadata, how='outer')

In [38]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1050 entries, 0 to 1049
Data columns (total 10 columns):
id                       1050 non-null object
full_date                873 non-null object
patient__autocolor       790 non-null float64
province__autocolor      873 non-null object
SEX__autocolor           873 non-null object
AGE__autocolor           872 non-null object
day                      873 non-null Int64
month                    873 non-null Int64
year                     873 non-null Int64
DEPARTMENT__autocolor    482 non-null object
dtypes: Int64(3), float64(1), object(6)
memory usage: 93.3+ KB


In [39]:
final_df.tail()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor
1045,20425209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2400
1046,20425676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRABAJADORES_OCT_NOV
1047,20440116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRABAJADORES_OCT_NOV
1048,20444230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRABAJADORES_OCT_NOV
1049,20284978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INGRESADO


In [40]:
def extract_lineage(sample):
    pango_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/pangolin', str(sample) + ".lineage.csv")
    if os.path.exists(pango_file):
        df = pd.read_csv(pango_file)
        return df.lineage[0]
    else:
        return None

In [41]:
final_df['Lineage__autocolor'] = final_df.apply(lambda x: extract_lineage(x['id']), axis=1)

In [42]:
def extract_EU(sample):
    eu_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user', str(sample) + ".tsv")
    if os.path.exists(eu_file):
        df = pd.read_csv(eu_file, sep="\t")
        if 'EU1EU2' in df.columns:
            eulist = df.EU1EU2.dropna().tolist()
        return (',').join([(x + "-" + str(eulist.count(x))) for x in set(eulist)])
    else:
        return None

In [43]:
final_df['EU1EU2__autocolor'] = final_df.apply(lambda x: extract_EU(x['id']), axis=1)

In [44]:
def extract_faulty(sample):
    sample_report = os.path.join(compare_folder, "report_samples.tsv")
    df = pd.read_csv(sample_report, sep="\t")
    df['sample'] = df['sample'].astype(str)
    if str(sample) in df['sample'].values:
        return round(df[df['sample'] == str(sample)]['faulty_frac'].values[0], 3)
    else:
        return None

In [45]:
final_df['FaultyFraction__autocolor'] = final_df.apply(lambda x: extract_faulty(x['id']), axis=1)

In [46]:
#query_dario = [20242010,20242093,20242702,20247604,20247878,20250481,20250983,20253374,20253475,20253522,20253595,20255027,20255086,20255313,20255524,20255685,20255915,20255928,20258070,20258214,20258238,20258322,20258440,20261662,20263252,20265521,20267856,20269026,20269341,20269791,20272101,20272242,20273792,20274440,20277245,20277326,20279174,20279180,20279189,20279243,20279283,20286667,20286864,20286872,20288207,20289563,20303453]
#query_dario = [str(x) for x in query_dario]


In [47]:
#final_df['Dario__autocolor'] = final_df.apply(lambda x: x['id'] in query_dario, axis=1)


In [48]:
before_june = final_df[final_df.month < 6]['id'].tolist()
before_june = [str(x) for x in before_june]
final_df['before_june_2020__autocolor'] = final_df.apply(lambda x: x['id'] in before_june, axis=1)

In [49]:
final_df = final_df.drop_duplicates(subset=['id'], keep="first")
out_file = os.path.join(compare_folder, 'wgs_microreact.csv')
final_df.to_csv(out_file, index=False)

In [50]:
final_df.head()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor,Lineage__autocolor,EU1EU2__autocolor,FaultyFraction__autocolor,before_june_2020__autocolor
0,20060161,09/03/2020,174.0,Madrid,H,84,9,3,2020,GRAVES,A,,0.004,True
1,20060178,09/03/2020,175.0,Madrid,H,76,9,3,2020,GRAVES,A,,0.004,True
2,20062236,10/03/2020,176.0,Madrid,M,96,10,3,2020,GRAVES,B.2,,0.001,True
3,20062563,10/03/2020,177.0,Madrid,H,78,10,3,2020,GRAVES,B.1.5,,0.002,True
4,20063795,12/03/2020,178.0,Madrid,H,85,12,3,2020,GRAVES,A.5,,0.001,True
